In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.alert import Alert
import re


In [2]:
def init():
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(5)
    return driver

In [3]:
def login(driver, id, pwd):
    
    interpark_login_url = 'https://ticket.interpark.com/Gate/TPLogin.asp?CPage=B&MN=Y&tid1=main_gnb&tid2=right_top&tid3=login&tid4=login&GPage=https%3A%2F%2Ftickets.interpark.com%2Fgoods%2F21002720'
    driver.get(interpark_login_url)

    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='leftLoginBox']/iframe[@title='login']"))
    driver.find_element_by_name('userId').send_keys(id)
    driver.find_element_by_name('userPwd').send_keys(pwd)
    driver.find_element_by_id('btn_login').click()
    
# 태그가 없으면 에러발생
def check_exists_by_element(driver, by, name):
    try:
        driver.find_element(by, name)
    except NoSuchElementException:
        return False
    return True

def close_tickting_info_popup(driver):
    # 예매안내가 팝업이 뜨면 닫기. ( ticketingInfo_check : True, False )
    ticketinginfo_check = check_exists_by_element(driver, By.XPATH, "//div[@class='popupWrap']/div[@class='popupFooter']/button[@class='popupCloseBtn is-bottomBtn']")
    if ticketinginfo_check:
        popup_close_btn = driver.find_element(By.XPATH, "//button[@class='popupCloseBtn is-bottomBtn']")
        if popup_close_btn.text != "":
            popup_close_btn.click()
    
def click_book_btn(driver):
    print(driver.find_element(By.XPATH, "//li[@data-view='month current']").text)
    
#     (driver.find_element(By.XPATH, "//li[@data-view='month next']").text)
    
    # 예매하기 버튼 클릭
    driver.find_element(By.XPATH, "//div[@class='sideBtnWrap']/a[@class='sideBtn is-primary']").click()

def switching_focus(driver):
    # 예매하기 눌러서 새창이 뜨면 포커스를 새창으로 변경
    driver.switch_to.window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])

def checkAlertMessageBox(driver):
    try:
        Alert(driver).dismiss()
    except:
        print("no alter")
    
def close_ticketing_check_popup(driver):
    try:
        # 예매안내가 팝업이 뜨면 닫기. ( ticketingInfo_check : True, False )
        driver.switch_to.default_content()
        ticketingInfo_check = check_exists_by_element(driver, By.XPATH, "//div[@class='layerWrap']/div[@class='titleArea']/a[@class='closeBtn']")
        if ticketingInfo_check:
            driver.find_element(By.XPATH, "//div[@class='layerWrap']/div[@class='titleArea']/a[@class='closeBtn']").click()
    except ElementNotInteractableException as ex:
        print("Occur ElementNotInteractableException")
        pass
    except Exception as ex:
        print("한 번 더! close ticketing check popup", ex)
        close_ticketing_check_popup(driver)
        
def clickWantDate(driver, wantYear, wantMonth, wantDate, wantTime):
    try:
        # 날짜 아이프레임
        driver.switch_to.default_content()
        driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
        # 월 체크
        calHead = driver.find_elements(By.XPATH, "//div[@class='calHead']/div[@class='month']/span")
        year_month = calHead[1].find_elements(By.XPATH, "//em")
        year = year_month[0].text  # 년
        month = year_month[1].text  # 월

        yearC = wantYear - int(year)	# wantYear : 예매 원하는 년
        monthC = wantMonth - int(month)	# wantMonth : 예매 원하는 월

        s = yearC * 12 + monthC
        i = 0
        # 월 이동
        if s > 0:
            while i < s:
                calHead[2].click()
                i = i + 1
                calHead = driver.find_elements(By.XPATH, "//div[@class='calHead']/div[@class='month']/span")
        elif s < 0:
            while i < s:
                calHead[0].click()
                i = i - 1
                calHead = driver.find_elements(By.XPATH, "//div[@class='calHead']/div[@class='month']/span")

        # 선택 가능한 날짜 모두 불러오기
        CellPlayDate = driver.find_elements(By.NAME, "CellPlayDate")

        # 일 선택
        for cell in CellPlayDate:
            if cell.text == wantDate:	# wantDate : 예매 원하는 일
                cell.click()
                break
        print(driver.find_element(By.XPATH, "//div[@class='month']").text)

        # 선택 가능한 날짜 모두 불러오기
        CellPlayDate = driver.find_elements(By.NAME, "CellPlayDate")

        # 일 선택
        for cell in CellPlayDate:
            if int(cell.text) == wantDate:	# wantDate : 예매 원하는 일
                cell.click()
                break

        CellPlaySeq = driver.find_elements(By.NAME, "CellPlaySeq")
        for cell in CellPlaySeq:
            time = cell.text[:2]+cell.text[4:6]
            print(time)
            if time == wantTime:
                cell.click()

        driver.switch_to.default_content()
        nextBtn = driver.find_element(By.XPATH, "//p[@id='LargeNextBtn']")
        print(nextBtn.text)
        nextBtn.click()
    except UnexpectedAlertPresentException as ex:
        print("Occur UnexpectedAlertPresentException Error! Try again")
        close_ticketing_check_popup(driver)
        clickWantDate(driver, wantYear, wantMonth, wantDate, wantTime)
    except Exception as ex:
        print("error 발생!", ex)
        clickWantDate(driver, wantYear, wantMonth, wantDate, wantTime)
#         print("한 번 더! clickWantDate", ex)
#         close_ticketing_check_popup(driver)
        
    
def clickWantSeat(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@id='divBookSeat']/iframe[@id='ifrmSeat']"))
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='seatL']/iframe[@id='ifrmSeatDetail']"))
    stySeats = driver.find_elements(By.XPATH, "//img[@class='stySeat']")
    for seat in stySeats:
        seatInfo = seat.get_attribute("onclick").split(',')

        floor = int(re.findall('\d+', seatInfo[2])[0])
        column = int(re.findall('\d+', seatInfo[3])[0])
        row = int(re.findall('\d+', seatInfo[4])[0])

        if floor == 1 and column <= 25 and 9 <= row <= 24:
            seat.click()
            break
    
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@id='divBookSeat']/iframe[@id='ifrmSeat']"))
    nextBtn = driver.find_element(By.XPATH, "//div[@class='btnWrap']/a")
    print(nextBtn.text)
    nextBtn.click()
        

In [19]:
checkAlertMessageBox(driver)

no alter


In [13]:
driver = init()
login(driver, 'dkrakzldnl', 'cjsdks12^^')
close_tickting_info_popup(driver)
click_book_btn(driver)
switching_focus(driver)
# close_ticketing_check_popup(driver)
# clickWantDate(driver, 2021, 8, 1, '1900')
# clickWantSeat(driver)
# clickTicketPrice(driver)
# confirmOderInfo(driver, '930111', ['010', '7294', '6010'])
# clickPaymentMethod(driver, '농협(중앙)')
# # agreePayment(driver)

2021. 07


In [94]:
close_ticketing_check_popup(driver)
clickWantDate(driver, 2021, 8, 1, '1900')
clickWantSeat(driver)
clickTicketPrice(driver)
confirmOderInfo(driver, '930111', ['010', '7294', '6010'])
clickPaymentMethod(driver, '농협(중앙)')
agreePayment(driver)

2021년 8월
1400
1900




NameError: name 'clickPaymentMethod' is not defined

In [28]:
clickWantSeat(driver)

In [4]:
def clickTicketPrice(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    ticketNums = driver.find_elements(By.XPATH, "//tr[@id='PriceRow001']/td[@class='taL']/select/option")
    for ticketNum in ticketNums:
        num = int(re.findall('\d+', ticketNum.text)[0])
        if num == 1:
            ticketNum.click()
            break

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()

In [5]:
birthDay = '930111'
phoneNumber = ['010', '7294', '6010']
def confirmOderInfo(driver, birthDay, phoneNumber):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    driver.find_element_by_id('YYMMDD').clear()
    driver.find_element_by_id('YYMMDD').send_keys(birthDay)
    driver.find_element_by_id('HpNo1').clear()
    driver.find_element_by_id('HpNo1').send_keys(phoneNumber[0])
    driver.find_element_by_id('HpNo2').clear()
    driver.find_element_by_id('HpNo2').send_keys(phoneNumber[1])
    driver.find_element_by_id('HpNo3').clear()
    driver.find_element_by_id('HpNo3').send_keys(phoneNumber[2])

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()

In [6]:
wantBank = '농협(중앙)'

def clickPaymentMethod(driver, wantBank):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    
    # 무통장입금 클릭(매칭 id로 검색)
    # 유연하게 진행하기 위해선 결제방식에 따른 id값을 미리 가지고 있을 필요있음
    driver.find_element(By.XPATH, "//tr[@id='Payment_22004']/td/input").click()

    # 입금 은행 - 농협 클릭
    banks = driver.find_elements(By.XPATH, "//select[@id='BankCode']/option")
    for bank in banks:
        if bank.text == wantBank:
            bank.click()
            break

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()


In [7]:

def agreePayment(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    driver.find_element_by_id('checkAll').click()

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='LargeNextBtn']").click()